In [3]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import glob
import random

In [4]:

#Haar Cascades
#usually the firs one works, in our case we try with upper body 
#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')
upperbody_cascade =  cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_upperbody.xml')
img = cv2.imread('C:/Users/Lucrezia/Downloads/Archive/src/frpr.jpg')#name of one image representing the party 
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
body = upperbody_cascade.detectMultiScale(gray, 1.3, 5)

for (x,y,w,h) in body:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    #eyes = eye_cascade.detectMultiScale(roi_gray) #if we want to find eyes in faces 
    #for (ex,ey,ew,eh) in eyes:
    #    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

cv2.imshow('img',img)
cv2.imwrite('Cascade.png', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
#Hough transform circles 
#how parameters work
'''
alpha 1  beta 0      --> no change  
0 < alpha < 1        --> lower contrast  
alpha > 1            --> higher contrast  
-127 < beta < +127   --> good range for brightness values
'''


img = cv2.imread('C:/Users/Lucrezia/Downloads/Archive/src/frpr.jpg')
img2 = cv2.imread('C:/Users/Lucrezia/Downloads/Archive/src/imout.jpg')
alpha = 3 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

output = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
#output=alpha*img+beta
#output = img.copy()

gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)

minDist = 150
param1 = 45 #500
param2 = 55 #200 #smaller value-> more false circles
minRadius = 2
maxRadius = 100 #10

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)

# ensure at least some circles were found
if circles is not None:
    
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(img, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(img, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    # show the output image
    cv2.imshow("output", img)
    cv2.waitKey(0)
else:
    print('nulla')
cv2.imwrite('Houghdetection.png', img)

cv2.destroyAllWindows()

In [8]:
#try to find a way to underline heads in the frame
img = cv2.imread('C:/Users/Lucrezia/Downloads/Archive/src/frpr.jpg')
img2 = cv2.imread('C:/Users/Lucrezia/Downloads/Archive/src/im_out.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2.shape
#img3=cv2.subtract(img,img2)
img3=img-img2
gray3=gray-gray2
ret,thresh1 = cv2.threshold(gray3,200,255,cv2.THRESH_BINARY)
ask3 = cv2.cvtColor(thresh1, cv2.COLOR_GRAY2BGR)
img4 = cv2.bitwise_or(img,ask3)
#img5 = cv2. cvtColor(img4, cv2. COLOR_GRAY2BGR)
cv2.imshow("output", img4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
#yolo detection 

In [10]:
# Load Yolo
# link fro the weights https://drive.google.com/file/d/1mcIlPcYjm34DdB7gA_uIZFw8x1-hAPSB/view?usp=sharing 
# link to the folder "heads" https://drive.google.com/drive/folders/1AkoVyfDcoRPSIDFVm3XJ4OGQ1siCpRsC?usp=sharing
 
net = cv2.dnn.readNet("yolov3_training_last29.weights", "yolov3_testing.cfg")

# Name custom object
classes = ["head"]

# Images path
yourPath=r"C:/Users/Lucrezia/Downloads/Archive/src/" # keep the r at the beginning
images_path = glob.glob(yourPath+"heads/*.jpg")


layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [11]:
# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
counter=0
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                #print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            #cv2.putText(img, label, (x, y + 30), font, 3, color, 2)
    if cv2.waitKey(10) == ord('q'):
        break
    cv2.imshow("Image", img)
    cv2.imwrite("Head"+str(counter)+".png",img)
    counter+=1
    key = cv2.waitKey(0)

cv2.destroyAllWindows()